In [106]:
%pip install ipython-autotime
%pip install pyspark

time: 11.2 s (started: 2023-09-13 18:32:52 +00:00)


In [107]:
%%script echo 'ignore cell'
import os
import sys

# resolver problema de versão python x pyspark workers
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

ignore cell
time: 6.07 ms (started: 2023-09-13 18:33:03 +00:00)


In [108]:
from google.colab          import drive, files
from pyspark.sql           import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types     import *

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 840 µs (started: 2023-09-13 18:33:03 +00:00)


In [109]:
drive.mount('/content/drive', force_remount=True)
spark = SparkSession.builder.master('local').appName('pyspark_app').getOrCreate()
spark

Mounted at /content/drive


time: 4.15 s (started: 2023-09-13 18:33:03 +00:00)


In [110]:
df = spark.read.csv("/content/drive/MyDrive/datasets/housing/housing.csv", header=True, inferSchema=True, encoding='utf-8')
print(type(df))
df.show(5)

<class 'pyspark.sql.dataframe.DataFrame'>
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NE

In [111]:
print(df.columns)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity']
time: 11.1 ms (started: 2023-09-13 18:33:08 +00:00)


In [112]:
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)

time: 6.82 ms (started: 2023-09-13 18:33:08 +00:00)


NaN verification

In [113]:
for col in df.columns:
  mask = df['longitude'].isNull()
  nan_amount = df.filter(mask).count()
  print(f'{col}: {nan_amount}')

longitude: 0
latitude: 0
housing_median_age: 0
total_rooms: 0
total_bedrooms: 0
population: 0
households: 0
median_income: 0
median_house_value: 0
ocean_proximity: 0
time: 6.09 s (started: 2023-09-13 18:33:08 +00:00)


Columns renaming

In [114]:
upper = [col.upper() for col in df.columns]
for col, up in zip(df.columns, upper):
  df = df.withColumnRenamed(col, up)
print(df.columns)

['LONGITUDE', 'LATITUDE', 'HOUSING_MEDIAN_AGE', 'TOTAL_ROOMS', 'TOTAL_BEDROOMS', 'POPULATION', 'HOUSEHOLDS', 'MEDIAN_INCOME', 'MEDIAN_HOUSE_VALUE', 'OCEAN_PROXIMITY']
time: 137 ms (started: 2023-09-13 18:33:14 +00:00)


Going back to the original format.

In [115]:
lower = [col.lower() for col in df.columns]
for col, low in zip(df.columns, lower):
  df = df.withColumnRenamed(col, low)
print(df.columns)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity']
time: 222 ms (started: 2023-09-13 18:33:14 +00:00)


Selecting columns

In [116]:
print(type(df.select(['longitude', 'latitude', 'households'])))
df.select(['longitude', 'latitude', 'households']).show(5)

<class 'pyspark.sql.dataframe.DataFrame'>
+---------+--------+----------+
|longitude|latitude|households|
+---------+--------+----------+
|  -122.23|   37.88|     126.0|
|  -122.22|   37.86|    1138.0|
|  -122.24|   37.85|     177.0|
|  -122.25|   37.85|     219.0|
|  -122.25|   37.85|     259.0|
+---------+--------+----------+
only showing top 5 rows

time: 233 ms (started: 2023-09-13 18:33:14 +00:00)
